In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost.sklearn import XGBRegressor
import joblib

In [2]:
def createdf(number):
    nombres_columnas = ['unit', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3']
    nombres_columnas += [f'sensor_{i}' for i in range(1, 24)]
    df = pd.read_csv(rf"./CMAPSSData/train_FD00{number}.txt",sep = " ", header = None, index_col = None)
    df.columns = nombres_columnas
    df = df.iloc[:, :-2]
    return df

In [3]:
df = createdf(4)

In [4]:
max_cycle_by_unit = df.groupby('unit')['cycle'].transform('max')
df['RUL'] = max_cycle_by_unit - df['cycle']

In [5]:
df.head()

,unit,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316


In [6]:
X_total = df.drop(['unit', 'cycle', 'RUL'], axis = 1)
y = df.RUL

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_total)
joblib.dump(scaler, 'scaler.joblib')
X_scaled = pd.DataFrame(X_scaled,columns=X_total.columns)
X_scaled.to_csv("X_scaled.csv", index= False)
X_scaled.head()

,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1.218156,0.864668,0.418783,-1.054690,-0.796416,-0.701412,-0.745729,-1.137677,-1.081831,-0.993802,...,-0.989007,0.417814,0.081921,0.063831,-0.694278,-0.638665,-0.114203,0.418783,-1.030999,-1.031756
1,-0.270478,0.414718,0.418783,0.692508,0.713666,0.562449,0.298212,0.363906,0.371152,0.332051,...,0.331131,0.415786,-0.253086,-0.125677,-0.694278,0.476120,0.655708,0.418783,0.352814,0.358264
2,1.218082,0.867565,0.418783,-1.054690,-0.815965,-0.704332,-0.711202,-1.137677,-1.083668,-0.988219,...,-0.990162,0.417658,-0.013912,0.153387,-0.694278,-0.674626,-0.114203,0.418783,-1.045089,-1.022649
3,1.217824,0.864668,0.418783,-1.054690,-0.822660,-0.722040,-0.702990,-1.137677,-1.081831,-0.989581,...,-0.988862,0.418048,0.096162,0.068362,-0.694278,-0.710586,-0.114203,0.418783,-1.039051,-1.023269
4,0.068094,0.158844,-2.387873,-0.391216,-1.160079,-1.532181,-1.410627,-0.270955,-0.475656,-0.738762,...,-0.741097,-2.389666,-2.358027,2.066982,-0.694278,-1.537685,-2.155843,-2.387873,-0.687814,-0.644612


In [8]:
X_important = X_scaled[['sensor_14','sensor_11','sensor_16','sensor_4', 'sensor_17', 'sensor_3', 'sensor_9']]
X_important.head()

,sensor_14,sensor_11,sensor_16,sensor_4,sensor_17,sensor_3,sensor_9
0,0.081921,-0.365205,-0.694278,-0.745729,-0.638665,-0.701412,-0.633237
1,-0.253086,0.328498,-0.694278,0.298212,0.476120,0.562449,0.560738
2,-0.013912,-0.374454,-0.694278,-0.711202,-0.674626,-0.704332,-0.646979
3,0.096162,-0.368288,-0.694278,-0.702990,-0.710586,-0.722040,-0.630180
4,-2.358027,-1.971512,-0.694278,-1.410627,-1.537685,-1.532181,-1.572260


In [9]:
X_descartes = X_scaled[['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1',
                       'sensor_2', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8',
                       'sensor_10', 'sensor_12', 'sensor_13', 'sensor_15', 'sensor_18',
                       'sensor_19', 'sensor_20', 'sensor_21']]
print(len(X_descartes.columns))
X_descartes.head()

17


,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_5,sensor_6,sensor_7,sensor_8,sensor_10,sensor_12,sensor_13,sensor_15,sensor_18,sensor_19,sensor_20,sensor_21
0,1.218156,0.864668,0.418783,-1.054690,-0.796416,-1.137677,-1.081831,-0.993802,-0.115765,-0.677047,-0.989007,0.417814,0.063831,-0.114203,0.418783,-1.030999,-1.031756
1,-0.270478,0.414718,0.418783,0.692508,0.713666,0.363906,0.371152,0.332051,0.653429,-0.207120,0.331131,0.415786,-0.125677,0.655708,0.418783,0.352814,0.358264
2,1.218082,0.867565,0.418783,-1.054690,-0.815965,-1.137677,-1.083668,-0.988219,-0.115352,-0.677047,-0.990162,0.417658,0.153387,-0.114203,0.418783,-1.045089,-1.022649
3,1.217824,0.864668,0.418783,-1.054690,-0.822660,-1.137677,-1.081831,-0.989581,-0.115627,-0.598726,-0.988862,0.418048,0.068362,-0.114203,0.418783,-1.039051,-1.023269
4,0.068094,0.158844,-2.387873,-0.391216,-1.160079,-0.270955,-0.475656,-0.738762,-2.156673,-1.303616,-0.741097,-2.389666,2.066982,-2.155843,-2.387873,-0.687814,-0.644612


In [10]:
pca = PCA(n_components=3)
X_reduced_data = pca.fit_transform(X_descartes)
joblib.dump(pca,'pca.pkl')
X_reduced = pd.DataFrame(
        X_reduced_data, 
        columns=[f'pca_{i+1}' for i in range(X_reduced_data.shape[1])],
        index=X_scaled.index  
    )
print(X_important.shape)
print(X_reduced.shape)

X_final = pd.concat([X_important, X_reduced], axis=1)
X_final.to_csv("X_final.csv", index= False)
X_final.describe()

(61249, 7)
(61249, 3)


,sensor_14,sensor_11,sensor_16,sensor_4,sensor_17,sensor_3,sensor_9,pca_1,pca_2,pca_3
count,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04,6.124900e+04
mean,-2.864027e-15,-1.583057e-15,7.760993e-17,1.374937e-15,1.450691e-16,-7.118449e-16,7.849160e-16,-3.707644e-16,-2.807415e-17,1.009277e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,3.538073e+00,2.044034e+00,5.086515e-01
min,-2.591715e+00,-2.107169e+00,-6.942780e-01,-1.487475e+00,-1.645567e+00,-1.650485e+00,-1.603216e+00,-4.378414e+00,-4.267027e+00,-7.870899e-01
25%,-6.048584e-02,-3.436230e-01,-6.942780e-01,-6.907542e-01,-6.386649e-01,-6.343475e-01,-6.057236e-01,-2.752226e+00,-1.101192e+00,-2.258086e-01
50%,1.867423e-01,-1.678850e-01,-6.942780e-01,-5.446845e-01,-4.948217e-01,-4.729975e-01,-4.805621e-01,-1.814866e+00,5.103578e-01,-7.523366e-02
75%,7.066452e-01,7.231374e-01,1.440345e+00,8.439411e-01,7.278456e-01,7.490426e-01,7.496530e-01,3.051229e+00,1.910324e+00,5.246782e-02
max,2.262619e+00,1.691238e+00,1.440345e+00,2.001688e+00,1.842630e+00,1.837708e+00,1.994916e+00,6.340555e+00,2.056422e+00,1.207658e+00


In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [12]:
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final, y, test_size=0.2, random_state=42)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
def modelo_XGBoost(X_train, y_train,X_test, y_test, label = 'final'):
    start = time.time()
    parameters = {
        'n_estimators': [  900, 1100, 1300],
        'max_depth': [7, 9, 11],
        'learning_rate': [0.01, 0.05, 0.1],
        'objective' :['reg:squarederror']
    }
    
    scorer = make_scorer(rmse, greater_is_better=False)
    
    xgb_model = XGBRegressor()
    kfold = KFold(n_splits=2, shuffle=False)
    
    grid_search = GridSearchCV(estimator=xgb_model,
                               param_grid=parameters,
                               cv=kfold,
                               scoring=scorer,
                               verbose=1,
                               n_jobs=-1)

    grid_search.fit(X_train, y_train)

    
    print("Best parameters found: ", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    joblib.dump(best_model, f'best_model_XGBoost_{label}.joblib')
    
    y_pred = best_model.predict(X_test)
    rmserror = mean_squared_error(y_test, y_pred)
    end = time.time()
    total_time = end - start
    return rmserror, total_time, best_model

In [14]:
%%time
modelo_XGBoost(X_train_final,y_train_final,X_test_final,y_test_final)

Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best parameters found:  {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 900, 'objective': 'reg:squarederror'}
CPU times: total: 19.3 s
Wall time: 1min 33s


(3428.40673828125,
 93.42394185066223,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=900,
              n_jobs=None, num_parallel_tree=None, ...))

In [15]:
modelo_XGBoost(X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled, label = 'scaled')

Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best parameters found:  {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1100, 'objective': 'reg:squarederror'}


(3044.06787109375,
 171.8362467288971,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1100,
              n_jobs=None, num_parallel_tree=None, ...))

In [16]:
def grid_search_random_forest(X_train, y_train,X_test, y_test, label='final'):
    start = time.time()
    parameters = {
        'n_estimators': [300, 600, 900],
        'max_depth': [11, 13, 15],
        'min_samples_split': [4, 6, 8]
    }

    scorer = make_scorer(rmse, greater_is_better=False)
    
    rf_model = RandomForestRegressor()

    kfold = KFold(n_splits=2, shuffle=False)

    grid_search = GridSearchCV(estimator=rf_model,
                               param_grid=parameters,
                               cv=kfold,  
                               scoring=scorer,
                               verbose=1,
                               n_jobs=-1)


    grid_search.fit(X_train, y_train)


    
    best_model = grid_search.best_estimator_
    joblib.dump(best_model, f'best_model_random_forest_{label}.joblib')
    y_pred = best_model.predict(X_test)
    rmserror = mean_squared_error(y_test, y_pred)
    end = time.time()
    total_time = end - start
    return rmserror, total_time, best_model


In [17]:
grid_search_random_forest(X_train_final,y_train_final,X_test_final,y_test_final)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


(3424.4605296997524,
 916.748655796051,
 RandomForestRegressor(max_depth=15, min_samples_split=8, n_estimators=600))

In [18]:
grid_search_random_forest(X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled, label = 'scaled')

Fitting 2 folds for each of 27 candidates, totalling 54 fits


(3061.6471566687346,
 1463.4549231529236,
 RandomForestRegressor(max_depth=15, min_samples_split=8, n_estimators=900))